In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv')

In [4]:
df

,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors,fuel_efficiency_mpg
0,170,3.0,159.0,3413.433759,17.7,2003,Europe,Gasoline,All-wheel drive,0.0,13.231729
1,130,5.0,97.0,3149.664934,17.8,2007,USA,Gasoline,Front-wheel drive,0.0,13.688217
2,170,NaN,78.0,3079.038997,15.1,2018,Europe,Gasoline,Front-wheel drive,0.0,14.246341
3,220,4.0,NaN,2542.392402,20.2,2009,USA,Diesel,All-wheel drive,2.0,16.912736
4,210,1.0,140.0,3460.870990,14.4,2009,Europe,Gasoline,All-wheel drive,2.0,12.488369
...,...,...,...,...,...,...,...,...,...,...,...
9699,140,5.0,164.0,2981.107371,17.3,2013,Europe,Diesel,Front-wheel drive,NaN,15.101802
9700,180,NaN,154.0,2439.525729,15.0,2004,USA,Gasoline,All-wheel drive,0.0,17.962326
9701,220,2.0,138.0,2583.471318,15.1,2008,USA,Diesel,All-wheel drive,-1.0,17.186587
9702,230,4.0,177.0,2905.527390,19.4,2011,USA,Diesel,Front-wheel drive,1.0,15.331551


In [6]:
colonnes = ['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year']
valeurs_manquantes = df[colonnes].isnull().sum()
valeurs_manquantes

engine_displacement      0
horsepower             708
vehicle_weight           0
model_year               0
dtype: int64

In [7]:
median_horsepower = df['horsepower'].median()
median_horsepower 

import numpy as np
from sklearn.model_selection import train_test_split

# Filtrer les colonnes demandées
colonnes = ['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year', 'fuel_efficiency_mpg']
df = df[colonnes]

# Supprimer les lignes avec des valeurs manquantes dans horsepower
df = df.dropna(subset=['horsepower'])

# Mélanger avec seed 42
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Diviser en train/val/test (60%/20%/20%)
train_size = 0.6
val_size = 0.2
test_size = 0.2

# Premier split : séparer train du reste
df_train, df_temp = train_test_split(df_shuffled, test_size=(val_size + test_size), random_state=42)

# Deuxième split : séparer val et test
df_val, df_test = train_test_split(df_temp, test_size=0.5, random_state=42)

print(f"Train: {len(df_train)} échantillons ({len(df_train)/len(df)*100:.1f}%)")
print(f"Validation: {len(df_val)} échantillons ({len(df_val)/len(df)*100:.1f}%)")
print(f"Test: {len(df_test)} échantillons ({len(df_test)/len(df)*100:.1f}%)")

Train: 5397 échantillons (60.0%)
Validation: 1799 échantillons (20.0%)
Test: 1800 échantillons (20.0%)


In [8]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

colonnes = ['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year', 'fuel_efficiency_mpg']
df = df[colonnes]

# Mélanger avec seed 42
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Diviser en train/val/test (60%/20%/20%)
train_size = 0.6
val_size = 0.2
test_size = 0.2

df_train, df_temp = train_test_split(df_shuffled, test_size=(val_size + test_size), random_state=42)
df_val, df_test = train_test_split(df_temp, test_size=0.5, random_state=42)

print(f"Valeurs manquantes dans horsepower - Train: {df_train['horsepower'].isnull().sum()}")
print(f"Valeurs manquantes dans horsepower - Validation: {df_val['horsepower'].isnull().sum()}")

# Calculer la moyenne sur l'ensemble d'entraînement seulement
mean_horsepower_train = df_train['horsepower'].mean()

# Option 1 : Remplir avec 0
df_train_zero = df_train.copy()
df_val_zero = df_val.copy()

df_train_zero['horsepower'] = df_train_zero['horsepower'].fillna(0)
df_val_zero['horsepower'] = df_val_zero['horsepower'].fillna(0)

# Option 2 : Remplir avec la moyenne (calculée sur train seulement)
df_train_mean = df_train.copy()
df_val_mean = df_val.copy()

df_train_mean['horsepower'] = df_train_mean['horsepower'].fillna(mean_horsepower_train)
df_val_mean['horsepower'] = df_val_mean['horsepower'].fillna(mean_horsepower_train)

# Préparer les features et target
features = ['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year']

# Option 1 : Modèle avec remplissage par 0
X_train_zero = df_train_zero[features]
y_train_zero = df_train_zero['fuel_efficiency_mpg']
X_val_zero = df_val_zero[features]
y_val_zero = df_val_zero['fuel_efficiency_mpg']

model_zero = LinearRegression()
model_zero.fit(X_train_zero, y_train_zero)
y_pred_zero = model_zero.predict(X_val_zero)
rmse_zero = np.sqrt(mean_squared_error(y_val_zero, y_pred_zero))

# Option 2 : Modèle avec remplissage par moyenne
X_train_mean = df_train_mean[features]
y_train_mean = df_train_mean['fuel_efficiency_mpg']
X_val_mean = df_val_mean[features]
y_val_mean = df_val_mean['fuel_efficiency_mpg']

model_mean = LinearRegression()
model_mean.fit(X_train_mean, y_train_mean)
y_pred_mean = model_mean.predict(X_val_mean)
rmse_mean = np.sqrt(mean_squared_error(y_val_mean, y_pred_mean))

print(f"\nRMSE avec remplissage par 0: {round(rmse_zero, 2)}")
print(f"RMSE avec remplissage par moyenne: {round(rmse_mean, 2)}")
print(f"Moyenne de horsepower (train): {round(mean_horsepower_train, 2)}")

# Déterminer la meilleure option
if rmse_zero < rmse_mean:
    meilleure_option = "Avec 0"
elif rmse_mean < rmse_zero:
    meilleure_option = "Avec la moyenne"
else:
    meilleure_option = "Les deux sont tout aussi bons"

print(f"\nMeilleure option: {meilleure_option}")

Valeurs manquantes dans horsepower - Train: 0
Valeurs manquantes dans horsepower - Validation: 0

RMSE avec remplissage par 0: 0.45
RMSE avec remplissage par moyenne: 0.45
Moyenne de horsepower (train): 149.37

Meilleure option: Les deux sont tout aussi bons


In [11]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

colonnes = ['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year', 'fuel_efficiency_mpg']
df = df[colonnes]

# Mélanger avec seed 42
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Diviser en train/val/test (60%/20%/20%)
df_train, df_temp = train_test_split(df_shuffled, test_size=0.4, random_state=42)
df_val, df_test = train_test_split(df_temp, test_size=0.5, random_state=42)

# Remplir les NA avec 0 (comme demandé)
df_train = df_train.fillna(0)
df_val = df_val.fillna(0)

# Préparer les features et target
features = ['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year']
X_train = df_train[features]
y_train = df_train['fuel_efficiency_mpg']
X_val = df_val[features]
y_val = df_val['fuel_efficiency_mpg']

# Essayer différentes valeurs de r (alpha dans Ridge)
r_values = [0, 0.01, 0.1, 1, 5, 10, 100]
best_rmse = float('inf')
best_r = None

print("RMSE pour différentes valeurs de r:")
print("-" * 30)

for r in r_values:
    # Entraîner le modèle Ridge
    model = Ridge(alpha=r, random_state=42)
    model.fit(X_train, y_train)
    
    # Prédire sur l'ensemble de validation
    y_pred = model.predict(X_val)
    
    # Calculer RMSE
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    rmse_rounded = round(rmse, 2)
    
    print(f"r = {r:6} -> RMSE = {rmse_rounded}")
    
    # Vérifier si c'est le meilleur RMSE (avec la condition corrigée)
    if rmse_rounded < best_rmse:
        best_rmse = rmse_rounded
        best_r = r
    elif rmse_rounded == best_rmse and r < best_r:
        # Si même RMSE, prendre le plus petit r
        best_r = r

print("-" * 30)
print(f"Meilleur r: {best_r}")
print(f"Meilleur RMSE: {best_rmse}")

RMSE pour différentes valeurs de r:
------------------------------
r =      0 -> RMSE = 0.51
r =   0.01 -> RMSE = 0.51
r =    0.1 -> RMSE = 0.51
r =      1 -> RMSE = 0.51
r =      5 -> RMSE = 0.51
r =     10 -> RMSE = 0.51
r =    100 -> RMSE = 0.51
------------------------------
Meilleur r: 0
Meilleur RMSE: 0.51


In [12]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

colonnes = ['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year', 'fuel_efficiency_mpg']
df = df[colonnes]

seeds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
rmse_scores = []

print("RMSE pour différentes seeds:")
print("-" * 30)

for seed in seeds:
    # Mélanger avec la seed courante
    df_shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)
    
    # Diviser en train/val/test (60%/20%/20%)
    df_train, df_temp = train_test_split(df_shuffled, test_size=0.4, random_state=seed)
    df_val, df_test = train_test_split(df_temp, test_size=0.5, random_state=seed)
    
    # Remplir les NA avec 0
    df_train = df_train.fillna(0)
    df_val = df_val.fillna(0)
    
    # Préparer les features et target
    features = ['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year']
    X_train = df_train[features]
    y_train = df_train['fuel_efficiency_mpg']
    X_val = df_val[features]
    y_val = df_val['fuel_efficiency_mpg']
    
    # Entraîner le modèle sans régularisation
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Prédire et calculer RMSE
    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    rmse_scores.append(rmse)
    
    print(f"Seed {seed} -> RMSE = {rmse:.3f}")

# Calculer l'écart type
std_rmse = np.std(rmse_scores)

print("-" * 30)
print(f"Scores RMSE: {[round(score, 3) for score in rmse_scores]}")
print(f"Écart type des scores RMSE: {std_rmse:.3f}")
print(f"Écart type arrondi à 3 décimales: {round(std_rmse, 3)}")

# Afficher les statistiques descriptives
print(f"\nMoyenne des RMSE: {np.mean(rmse_scores):.3f}")
print(f"Min RMSE: {np.min(rmse_scores):.3f}")
print(f"Max RMSE: {np.max(rmse_scores):.3f}")

RMSE pour différentes seeds:
------------------------------
Seed 0 -> RMSE = 0.518
Seed 1 -> RMSE = 0.509
Seed 2 -> RMSE = 0.514
Seed 3 -> RMSE = 0.515
Seed 4 -> RMSE = 0.518
Seed 5 -> RMSE = 0.524
Seed 6 -> RMSE = 0.515
Seed 7 -> RMSE = 0.525
Seed 8 -> RMSE = 0.508
Seed 9 -> RMSE = 0.529
------------------------------
Scores RMSE: [np.float64(0.518), np.float64(0.509), np.float64(0.514), np.float64(0.515), np.float64(0.518), np.float64(0.524), np.float64(0.515), np.float64(0.525), np.float64(0.508), np.float64(0.529)]
Écart type des scores RMSE: 0.007
Écart type arrondi à 3 décimales: 0.007

Moyenne des RMSE: 0.518
Min RMSE: 0.508
Max RMSE: 0.529


In [13]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

colonnes = ['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year', 'fuel_efficiency_mpg']
df = df[colonnes]

# Utiliser la seed 9
seed = 9

# Mélanger avec seed 9
df_shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)

# Diviser en train/val/test (60%/20%/20%)
df_train, df_temp = train_test_split(df_shuffled, test_size=0.4, random_state=seed)
df_val, df_test = train_test_split(df_temp, test_size=0.5, random_state=seed)

# Combiner train et validation
df_train_val = pd.concat([df_train, df_val], ignore_index=True)

# Remplir les NA avec 0
df_train_val = df_train_val.fillna(0)
df_test = df_test.fillna(0)

# Préparer les features et target
features = ['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year']
X_train_val = df_train_val[features]
y_train_val = df_train_val['fuel_efficiency_mpg']
X_test = df_test[features]
y_test = df_test['fuel_efficiency_mpg']

# Entraîner le modèle avec r=0.001 (Ridge regression)
model = Ridge(alpha=0.001, random_state=seed)
model.fit(X_train_val, y_train_val)

# Prédire sur l'ensemble de test
y_pred_test = model.predict(X_test)

# Calculer RMSE sur test
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"Taille de l'ensemble train+val: {len(df_train_val)}")
print(f"Taille de l'ensemble test: {len(df_test)}")
print(f"RMSE sur l'ensemble de test: {rmse_test:.3f}")
print(f"RMSE arrondi à 3 décimales: {round(rmse_test, 3)}")

Taille de l'ensemble train+val: 7763
Taille de l'ensemble test: 1941
RMSE sur l'ensemble de test: 0.532
RMSE arrondi à 3 décimales: 0.532
